In [26]:
from PIL import ImageFont, ImageDraw, Image, ImageFilter
import random
import time
import math
import os
import json


In [27]:
config = json.load(open('config.json', 'r'))


In [28]:
# open the file all_moves.txt and read all the lines
moves = {}
with open(config['moves_distribution_file'], 'r') as f:
    moves = {a:int(b) for a,b in [line.strip().split(",") for line in f.readlines()]}
keys, values = zip(*moves.items())


def get_move(): 
    return random.choices(keys, weights=values)[0]




In [29]:

def dashed_line(draw, x1, y1, x2, y2, dash_length, dash_separation, fill="black", width=2):
    length = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    dash_count = int(length / (dash_length + dash_separation))

    # Calculate the gap between dashes
    dash_x = (x2 - x1) / dash_count
    dash_y = (y2 - y1) / dash_count

    for i in range(dash_count):
        # Calculate dash start and end points
        x_start = x1 + i * dash_x
        y_start = y1 + i * dash_y
        x_end = x_start + dash_length * (x2 - x1) / length
        y_end = y_start + dash_length * (y2 - y1) / length

        # Draw the dash
        draw.line([(x_start, y_start), (x_end, y_end)], fill=fill, width=width)
   

def draw_line(draw, x1, y1, x2, y2,style="solid",fill="black",width=2):
    if style=="solid":
        draw.line((x1, y1, x2, y2), fill=fill, width=width)
    elif style=="dashed":
        dashed_line(draw, x1, y1, x2, y2, dash_length=5, dash_separation=4, fill=fill, width=width)
    else:
        print("Unknown style")
        return




In [30]:
# for each font in the fonts folder, add the font path to the list
font_paths = []
for root, dirs, files in os.walk(config['fonts_folder']):
    for file in files:
        if file.upper().endswith(".TTF") and not "BOLD" in file.upper():
             font_paths.append(os.path.join(root, file))

print(len(font_paths))

359


In [31]:


text_colors = ["black","navy","red","green","blue","purple","brown"]
background_colors = ["white","lightgrey","beige","ivory"]
line_styles = ["solid","dashed"]
line_colors = ["black","grey"]


def create_box_from_text():
    
    text = get_move()
    font_path = random.choice(font_paths)
    font_size = random.randint(30, 51)
    print(f" {font_path} {font_size} {text}")
    font = ImageFont.truetype(font_path, size=font_size)
    rotation_angle = random.uniform(-1, 1)

    

    image_size = (int(random.gauss(180, 5)), int(random.gauss(80, 5)))



    background_color = random.choice(background_colors)

    # Create a larger canvas
    canvas_size = (300, 200)

    box_shift_x = (canvas_size[0] - image_size[0]) / 2
    box_shift_y = (canvas_size[1] - image_size[1]) / 2
    
    img = Image.new("RGB", canvas_size, background_color)

    rectangle_width = random.gauss(130, 5)
    rectangle_height = random.gauss(52, 3)

    

    draw = ImageDraw.Draw(img)

    draw_point = (random.gauss(box_shift_x+(image_size[0]-rectangle_width)/2,3), random.gauss(box_shift_y+(image_size[1]-rectangle_height)/2,3))
    bottom_right = (draw_point[0] + rectangle_width, draw_point[1] + rectangle_height)


    line_style = random.choice(line_styles)
    line_color = random.choice(line_colors)
    vertical_line_color = random.choice([line_color,background_color])
    #width should be an integer
    line_width_horizontal = int(random.gauss(2,1))
    line_width_vertical = int(random.gauss(2,1)) if vertical_line_color == line_color else int(random.gauss(8,1))


    has_cell_top = random.random() < 0.8
    has_cell_bottom = random.random() < 0.8
    has_cell_left = random.random() < 0.8
    has_cell_right = random.random() < 0.8

    draw_horizontal_left = 0 if has_cell_left else draw_point[0]
    draw_horizontal_right = canvas_size[0] if has_cell_right else bottom_right[0]

    draw_vertical_top = 0 if has_cell_top else draw_point[1]
    draw_vertical_bottom = canvas_size[1] if has_cell_bottom else bottom_right[1] 


    # draw the lines of the rectangle one by one
    draw_line(draw, draw_horizontal_left, draw_point[1], draw_horizontal_right, draw_point[1],style=line_style,fill=line_color,width=line_width_horizontal)
    draw_line(draw, draw_horizontal_left, bottom_right[1], draw_horizontal_right, bottom_right[1],style=line_style,fill=line_color,width=line_width_horizontal)
    
    # draw this line 80% of the time
    if random.random() < 0.8:
        draw_line(draw, draw_point[0], draw_vertical_top, draw_point[0], draw_vertical_bottom,style=line_style,fill=vertical_line_color,width=line_width_vertical)
        
    if random.random() < 0.8:
        draw_line(draw, bottom_right[0], draw_vertical_top, bottom_right[0], draw_vertical_bottom,style=line_style,fill=vertical_line_color,width=line_width_vertical)

        
    # the text should be centered in the box with some random shift
    shift_x = random.gauss(0, 0.05)*rectangle_width
    shift_y = random.gauss(0, 0.05)*rectangle_height
    
    text_x = (rectangle_width)/2 + draw_point[0] + shift_x
    text_y = (rectangle_height)*0.9 + draw_point[1] + shift_y
    

    
    text_point = (text_x, text_y)
    text_color = random.choice(text_colors)


    draw.text(text_point, text, font=font,anchor="ms", fill=text_color)

    # THIS PART WAS FOR TESSERACT

    # image_bottom_in_canvas = (canvas_size[1] - image_size[1]) / 2 + image_size[1]
    # boxxes = []
    # top_left_x, _, _, _ = draw.textbbox(text_point, text, font=font,anchor="ms")
    # for i in range(len(text)):
    #     right = top_left_x + font.getmask(text[:i+1]).size[0]
    #     (_,top,_,_) = draw.textbbox((right,text_y), text[i], font=font,anchor="rs")
    #     left = right - font.getmask(text[i]).size[0]  
    #     letter_size = font.getmask(text[i]).size
    #     box = (left-box_shift_x,max(0,top)-box_shift_y,right-box_shift_x,min(image_bottom_in_canvas,top+letter_size[1])-box_shift_y)
    #     boxxes += [box]
        


        
    

    if has_cell_top :
        draw.text((text_x, text_y-rectangle_height), get_move(), font=font,anchor="ms", fill=text_color)
    if has_cell_bottom :
        draw.text((text_x, text_y+rectangle_height), get_move(), font=font,anchor="ms", fill=text_color)
    if has_cell_left :
        draw.text((text_x-rectangle_width, text_y), get_move(), font=font,anchor="ms", fill=text_color)
    if has_cell_right :
        draw.text((text_x+rectangle_width, text_y), get_move(), font=font,anchor="ms", fill=text_color)
    
    if has_cell_top and has_cell_left:
        draw.text((text_x-rectangle_width, text_y-rectangle_height), get_move(), font=font,anchor="ms", fill=text_color)
    if has_cell_top and has_cell_right:
        draw.text((text_x+rectangle_width, text_y-rectangle_height), get_move(), font=font,anchor="ms", fill=text_color)
    if has_cell_bottom and has_cell_left:
        draw.text((text_x-rectangle_width, text_y+rectangle_height), get_move(), font=font,anchor="ms", fill=text_color)
    if has_cell_bottom and has_cell_right:
        draw.text((text_x+rectangle_width, text_y+rectangle_height), get_move(), font=font,anchor="ms", fill=text_color)


    text_window = img.getbbox()
    img = img.crop(text_window)

    img = img.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.1, 1)))

    for _ in range(image_size[0] * image_size[1] // 10):  # Adjust the divisor to control noise density
        x = random.randint(0, image_size[0] - 1)
        y = random.randint(0, image_size[1] - 1)
        current_color = img.getpixel((x, y))
        noise_color = tuple([int(random.gauss(current_color[i], 10)) for i in range(3)])
        img.putpixel((x, y), noise_color)

    # Add random rotation
    img = img.rotate(rotation_angle, resample=Image.BICUBIC, expand=True)

    # Crop the rotated image to the original rectangle
    img = img.crop(((canvas_size[0]-image_size[0]) / 2, (canvas_size[1]-image_size[1]) / 2, (canvas_size[0]+image_size[0]) / 2, (canvas_size[1]+image_size[1]) / 2))

    return img,text


# Define your start and end values
start = config['start']
end = config['end']


folderPath = config['data_output_folder']

# create the output folder if it does not exist
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

def process_image(i):
    img, text = create_box_from_text()
    
    # img.show()
    img.save(os.path.join(folderPath, str(i) + ".png"))
    with open(os.path.join(folderPath, str(i) + ".txt"), "w") as f:
        f.write(text)
    
prev_time = time.time()
for i in range(start, end): 
    if i % 100 == 0:
        print(f"{(i - start) / (end - start) * 100:.2f}%")
        print(f"{time.time() - prev_time:.2f}s")
        prev_time = time.time()
    process_image(i)



with open(os.path.join(folderPath, "words.txt"), "w") as f:
    f.write("\n".join(moves.keys()))





0.00%
0.00s
 fonts/Rustic Pantry.ttf 38 Rh2
 fonts/BeautyHandwriting-Regular.ttf 37 Rb6
 fonts/Manly Signature Regular.ttf 50 Qxe6
 fonts/Milky.ttf 40 Kf2
 fonts/Goodnight-London-Sans.ttf 35 Qxa3
 fonts/Revitalis.ttf 47 Nxd7
 fonts/Sweet Romance.ttf 46 a4
 fonts/Kids Hand.ttf 34 Qh1
 fonts/Signatria.ttf 34 Bg1
 fonts/Skinny Flower Light.ttf 32 b1=Q+


In [32]:


x = 3
y = 8

images = []
for i in range(24):
    img, text = create_box_from_text()
    images.append(img)

widths, heights = zip(*(i.size for i in images))

total_width = (max(widths)+1) * x +1
total_height = (max(heights)+1) * y +1

new_im = Image.new("RGB", (total_width, total_height))

x_offset = 1
y_offset = 1
for i in range(24):
    new_im.paste(images[i], (x_offset, y_offset))
    x_offset += images[i].size[0] + 1
    if i % x == x-1:
        y_offset += max(heights)+1
        x_offset = 1




# new_im.show()




 fonts/Haritta.ttf 30 axb5
 fonts/Betterlett.ttf 47 Rg1
 fonts/Unicorn Giggles - TTF.ttf 45 Bxh3+
 fonts/BLACKBOYSONMOPEDS.ttf 47 fxg4
 fonts/Stay Wild.ttf 49 Qd7
 fonts/Naishila Dancing Script.ttf 47 Nfe5
 fonts/danielbk.ttf 41 Ka3
 fonts/Takipedey.ttf 50 Nh6
 fonts/GOODDP__.TTF 38 Ke6
 fonts/AydieFont.ttf 42 Bd2
 fonts/Mango Salsa - Personal Use.ttf 50 Rxe4
 fonts/Hesterica.ttf 51 Re4+
 fonts/AnsteryScript.ttf 38 f3
 fonts/Silver Pen.ttf 46 Re2+
 fonts/Milky.ttf 37 gxh6
 fonts/SMASH.ttf 49 Nxh2
 fonts/bromello-Regular.ttf 48 Rff1
 fonts/Callista.ttf 35 Qd8
 fonts/Silence Rocken.ttf 49 Rf7
 fonts/scb.ttf 42 Rg7+
 fonts/Stay Wild.ttf 44 Rc6+
 fonts/Primrose.ttf 41 Qa4
 fonts/I Love Glitter - TTF.ttf 42 Nf1
 fonts/Moonstar.ttf 36 Qe7
